In [1]:
from src.scraping.browser_managment import BrowserManager, ParallelBrowserManager
from src.scraping.google_scraping.google_init import GoogleInit


In [1]:
from src.scraping.google_scraping.review_data_saver import ReviewDataSaver
from src.scraping.google_scraping.review_extractor import ReviewExtractor
from src.scraping.google_scraping.review_loader import ReviewLoader
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.scraping.browser_managment.browser_manager import BrowserManager  # Ensure this is the updated BrowserManager
from src.scraping.browser_managment.parallel_browser_manager import ParallelBrowserManager
from selenium.webdriver.common.by import By
import time
from src.scraping.google_scraping.review_interaction_handler import ReviewInteractionHandler

def perform_search(query):
    # This function now initializes the BrowserManager inside to ensure fresh browser instances
    browser_manager = BrowserManager()
    review_handler = ReviewInteractionHandler(browser_manager.driver)
    review_loader = ReviewLoader(browser_manager.driver)  # Assuming ReviewLoader is properly imported
    review_extractor = ReviewExtractor(browser_manager.driver)  # Assuming ReviewExtractor is properly imported
    try:
        browser_manager.navigate_to_url(f"https://www.google.hr/search?q={query}")
        review_handler.accept_cookies()
        review_handler.navigate_to_reviews()
        review_loader.load_reviews()  # Load all reviews
        review_handler.click_more_buttons()  # Click all "More" buttons
        reviews_df = review_extractor.collect_reviews()  # Adjust based on actual return type/method
        saver = ReviewDataSaver(reviews_df, query)
        saver.save_reviews_to_csv()
        # Save reviews using the modified class/method

    
        # Calculate the ratio and percentage here
        loaded_reviews = review_loader.total_loaded_reviews
        scraped_reviews = review_extractor.total_scraped_reviews
        ratio = scraped_reviews / loaded_reviews if loaded_reviews > 0 else 0
        percentage = (scraped_reviews / loaded_reviews * 100) if loaded_reviews > 0 else 0
    
        # Print the metrics
        print(f"Query: {query}")
        print(f"Final count of reviews loaded: {loaded_reviews}, Scraped/Loaded Ratio: {scraped_reviews}/{loaded_reviews} = {ratio:.2f}, Success Percentage: {percentage:.2f}%")
    
        result = True  # Adjust this as needed based on your error handling
    except Exception as e:
        print(f"An error occurred during search for '{query}': {e}")
        result = False
        browser_manager.close_browser()
    finally:
        browser_manager.close_browser()
    return result

# Queries including the city
queries = ['Restoran Casablanca Zagreb', 'Restoran Potkova Zagreb', 'Restaurant Balon Zagreb']

# Initialize the ParallelBrowserManager with the desired number of workers
parallel_manager = ParallelBrowserManager(workers=3)

# Using list comprehension to create the args_list for each query
args_list = [(query,) for query in queries]

# Execute the searches in parallel
results = parallel_manager.perform_parallel_action(perform_search, args_list)

# Output the results
for query, result in zip(queries, results):
    print(f"Search for '{query}': {'Success' if result else 'Failure'}")


The chromedriver version (121.0.6167.85) detected in PATH at C:\Users\ivan.zeljeznjak\Desktop\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (122.0.6261.95); currently, chromedriver 122.0.6261.94 is recommended for chrome 122.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (121.0.6167.85) detected in PATH at C:\Users\ivan.zeljeznjak\Desktop\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (122.0.6261.95); currently, chromedriver 122.0.6261.94 is recommended for chrome 122.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (121.0.6167.85) detected in PATH at C:\Users\ivan.zeljeznjak\Desktop\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (122.0.6261.95); currently, chromedriver 122.0.6261.94 is recommended for chrome 122.*, so it is advised to delete the driver in PATH and retry


Found 692 reviews.
Found 2009 reviews.
Found 791 reviews.
Timeout waiting for the reviews to become clickable.
Found 692 reviews.
Failed to navigate to reviews: Message: element click intercepted: Element <a href="#" data-fid="0x476678824c73c52d:0x44de8eca72842acf" data-is_owner="false" data-sort_by="qualityScore" data-async-trigger="reviewDialog" role="button" jsaction="FNFY6c" data-ved="2ahUKEwiO0telzt-EAxVYHbkGHanfDPsQ8mt6BAhcEAY">...</a> is not clickable at point (1013, 427). Other element would receive the click: <div class="c9QyIf">...</div>
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF6B5395E42+3538674]
	(No symbol) [0x00007FF6B4FB4C02]
	(No symbol) [0x00007FF6B4E65AEB]
	(No symbol) [0x00007FF6B4EB33E4]
	(No symbol) [0x00007FF6B4EB1122]
	(No symbol) [0x00007FF6B4EAE884]
	(No symbol) [0x00007FF6B4EAD5A9]
	(No symbol) [0x00007FF6B4EA1950]
	(No symbol) [0x00007FF6B4ECF0AA]
	(No symbol) [0x00007FF6B4EA124A]
	(No symbol) [0x00007FF6B4ECF2C0]
	(No sy

In [2]:
#placeholder